# Risultati degli archivi

Questo notebook ispeziona automaticamente tutti i file `.zip` presenti nella directory `results`,
interpreta il nome del file per ottenere metadati utili ed estrae il contenuto di ciascun archivio
mostrandolo in chiaro qui sotto.


In [ ]:
from pathlib import Path
import zipfile
from IPython.display import display, Markdown

RESULTS_DIR = Path('results')
zip_paths = sorted(RESULTS_DIR.glob('*.zip'))

if not zip_paths:
    display(Markdown('> **Nessun archivio trovato nella directory `results`.**'))
else:
    display(Markdown(f'> Trovati **{len(zip_paths)}** archivi ZIP in `{RESULTS_DIR}`.'))


In [ ]:
def parse_zip_metadata(zip_path):
    """Estrae dataset, classe, stato e host dal nome del file zip."""
    dataset, class_name, is_completed, host_number = zip_path.stem.rsplit('_', 3)
    return {
        'zip_path': zip_path,
        'dataset': dataset,
        'class': class_name,
        'is_completed': is_completed,
        'host_number': host_number,
        'size_bytes': zip_path.stat().st_size,
    }

metadata = [parse_zip_metadata(path) for path in zip_paths]

if metadata:
    header = '| Dataset | Classe | Completato | Host | Dimensione (byte) | Archivio |\n'
    header += '|---------|--------|------------|------|-------------------|----------|\n'
    lines = []
    for item in metadata:
        lines.append(f"| {item['dataset']} | {item['class']} | {item['is_completed']} | {item['host_number']} | {item['size_bytes']:,} | {item['zip_path'].name} |")
    table = header + '\n'.join(lines)
    display(Markdown(table))
else:
    display(Markdown('> **Nessun archivio da mostrare.**'))


In [ ]:
def display_zip_contents(zip_path):
    meta = parse_zip_metadata(zip_path)
    display(Markdown(f"## {meta['dataset']} (classe {meta['class']}) — completato: `{meta['is_completed']}` — host: `{meta['host_number']}`"))
    with zipfile.ZipFile(zip_path) as archive:
        members = [m for m in archive.namelist() if not m.endswith('/')]
        if not members:
            display(Markdown('Nessun file all\'interno dell\'archivio.'))
            return
        for member in members:
            data = archive.read(member)
            try:
                text = data.decode('utf-8')
                display(Markdown(f"### {member}\n````text\n{text}\n````"))
            except UnicodeDecodeError:
                display(Markdown(f"### {member}\nFile binario ({len(data)} byte)."))

for path in zip_paths:
    display_zip_contents(path)
